# CaseLaw dataset to assist with Law-Research - EDA
---
<dl>
  <dt>Acquiring the dataset</dt>
  <dd>We initially use dataset of all cases in USA to be able to train it and as a proof of concept.</dd>
  <dd>The dataset is available in XML format, which we will put in mongodb or firebase format based on how unstructured the dataset is.</dd>
    <dd>dataset url: (https://case.law/)
</dd>

  <dt>Research</dt>
  <dd>We are looking into <em>NLP</em>, <em>LSTM</em> and <em>Sentiment Analysis</em>.</dd>
</dl>

In [129]:
import jsonlines
from pymongo import MongoClient

In [130]:
# client = MongoClient()
client = MongoClient()
db = client.legal_ai
cases = db.cases

In [9]:
some_date = '1820-01'

In [10]:
print(int(some_date[0:4])<1950)

True


In [86]:
id_saved = []
with jsonlines.open('../data.jsonl') as reader:
    for obj in reader:
        if int(obj['decision_date'][0:4])>1950:
            case_id = cases.insert_one(obj).inserted_id
            id_saved.append(case_id)

In [87]:
len(id_saved)

30075

## Testing out Similarity Mechanism
---
### Setup
- Test PyDictionary to build keywords
- Construct a mechanism, to extract keywords, and store in a searchable manner.
---
### Search
- Build keywords out of your search
- Search among dataset keywords
- Nearest dates, highest weight, highest precidence shows up
- Pagination scroll, continues the search.

In [ ]:
# NLTK


## Transforming dataset
---
### Extract the first data and study it
- Identify the key elements that need to be transformed & list them
- Build a mechanism to transform for one datapoint.
---
### Perform for entire dataset
- Run a loop and apply the same changes for every datapoints.

In [18]:
# Extracting the first element
first_case = cases.find_one()

In [20]:
import xml.etree.ElementTree as ET
root = ET.fromstring(first_case['casebody']['data'])

In [21]:
root

<Element '{http://nrs.harvard.edu/urn-3:HLS.Libr.US_Case_Law.Schema.Case_Body:v1}casebody' at 0x11232c2c8>

# Getting the case body cleaned into a seperate field on db
> 

In [71]:
summary=''
for child in root:
    for sub_child in child:
        if 'footnotemark' in sub_child.tag[sub_child.tag.index("}")+1:] or 'author' in sub_child.tag[sub_child.tag.index("}")+1:]:
            continue
        summary+=sub_child.text + "\n"

In [73]:
print(summary)

The executive secretary of the State Game and Fish, Commission declined to authorize payment of $105 to four persons who claimed to have killed seven wolves over six months old, as evidenced by certificate of the Boone County Court. See Act 183 of 1949. The statute is entitled “An Act to authorize . . . counties ... to pay bounties for the killing of wolves and to provide that the State . . . shall pay an equal sum as a bounty, and for other purposes.” The emergency clause is a finding that farmers are suffering irreparable damage ‘£ from wolves destroying cattle and other live stock.” The measure received two-thirds of the votes of all members elected to each branch of the General Assembly.
The complaint is a petition for mandamus alleging that Boone county has paid bounties on the basis of $20 for each wolf killed; but, § 2 of Act 183, the amount payable by the State cannot exceed $15.
The answer, among other defenses, asserts that T. H. McAmis as Secretary of the Game and Fish Commi

# Do the same for all the files now!

In [113]:
all_cases = cases.find()

In [104]:
all_cases.count()

/Users/flynsequeira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


30075

In [91]:
check_one = True
for each_case in all_cases:
    root = ET.fromstring(each_case['casebody']['data'])
    summary=''
    for child in root:
        for sub_child in child:
            if 'footnotemark' in sub_child.tag[sub_child.tag.index("}")+1:] or 'author' in sub_child.tag[sub_child.tag.index("}")+1:]:
                continue
            summary+=sub_child.text + "\n"
    myquery = { "_id": each_case['_id'] }
    newvalues = { "$set": { "summary": summary } }
    cases.update_one(myquery, newvalues)

checking for one
added to case check: 5c1549de08ef9543808eabd0
{'_id': ObjectId('5c1549de08ef9543808eabd0'), 'id': 1611964, 'name': 'Keith v. State', 'name_abbreviation': 'Keith v. State', 'decision_date': '1951-01-15', 'docket_number': '4641', 'first_page': '174', 'last_page': '181', 'citations': [{'type': 'official', 'cite': '218 Ark. 174'}, {'type': 'parallel', 'cite': '235 S.W.2d 539'}], 'volume': {'volume_number': '218'}, 'reporter': {'full_name': 'Arkansas Reports'}, 'court': {'id': 8808, 'name': 'Arkansas Supreme Court', 'name_abbreviation': 'Ark.', 'jurisdiction_url': None, 'slug': 'ark'}, 'jurisdiction': {'id': 34, 'slug': 'ark', 'name': 'Ark.', 'name_long': 'Arkansas', 'whitelisted': True}, 'casebody': {'data': '<?xml version=\'1.0\' encoding=\'utf-8\'?>\n<casebody xmlns="http://nrs.harvard.edu/urn-3:HLS.Libr.US_Case_Law.Schema.Case_Body:v1" xmlns:xlink="http://www.w3.org/1999/xlink" firstpage="174" lastpage="181"> <docketnumber id="b198-5" pgmap="198">4641</docketnumber> <ci

# Change Decision Date to mongodb date format

In [95]:
import datetime


In [114]:
count = 0
for each_case in all_cases:
    try:
        decision_date = datetime.datetime.strptime(each_case['decision_date'], "%Y-%m-%d %H:%M:%S")
    except:
        try:
            decision_date = datetime.datetime.strptime(each_case['decision_date'], "%Y-%m-%d")
        except:
            try:
                decision_date = datetime.datetime.strptime(each_case['decision_date'], "%Y-%m")
            except:
                try:
                    decision_date = datetime.datetime.strptime(each_case['decision_date'], "%Y")
                except:
                    pass
    myquery = { "_id": each_case['_id'] }
    newvalues = { "$set": { "decision_date": decision_date } }
    cases.update_one(myquery, newvalues)

1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00


1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00


1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00


1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00


1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00


...

1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00


1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00


1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00


1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00


1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00
1994-12-05 00:00:00


# Elastic Search

In [172]:
import elasticsearch
from datetime import datetime
date = datetime(1980, 1, 1)
es = elasticsearch.Elasticsearch()  # use default of localhost, port 9200


In [183]:
# Take only the latest cases
all_cases = cases.find({ "decision_date":{"$gte":date}})

In [184]:
all_cases.count()

/Users/flynsequeira/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


20778

In [185]:
id=1
for each_case in all_cases:
    new_data = { "id": str(each_case['_id']), "case_body": each_case['case_body'] }
    es.index(index='case_law', doc_type='case_body', id=id, body=new_data)
    id+=1

In [190]:
t

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 3802,
  'max_score': 9.243528,
  'hits': [{'_index': 'case_law',
    '_type': 'case_body',
    '_id': '9383',
    '_score': 9.243528,
    '_source': {'id': '5c15520f08ef954ad387c407',
     'case_body': 'Robert R. Morgan was employed as the CEO and chairman of the board of Clinton State Bank. The written contract provided that he would be employed for a period of five years beginning March 1, 1987. On October 15, 1989, Morgan was fired.\nOn January 29, 1990, some two years before the expiration of the employment contract, Morgan brought suit for damages for breach in Pulaski County Chancery Court. The chancellor held that Mr. Morgan was entitled to damages for breach incurred through the date of trial and awarded $89,000.00 for wrongful termination and directed the bank to repurchase Mr. Morgan’s home for $85,500.00 within thirty days of the date of entry of the judg

In [191]:
es.search('case_law', 'contract breach')

{'took': 1,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 0, 'max_score': None, 'hits': []}}

In [ ]:
es.get(index='case_law', doc_type='case_body'

In [ ]:
map([{'a':'b','c':'d'},{'a':'x','c':'y'}],)

In [193]:
print(''+'sldkfj')

sldkfj
